In [23]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown,display,update_display

In [26]:
load_dotenv(override=True)
openai_api_key=os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"openai api key starts with {openai_api_key[:8]}")
else:
    print(f"openai api key is not found")
if anthropic_api_key:
    print(f"anthropic api key starts with {anthropic_api_key[:4]}")
else:
    print(f"anthropic api key is not found")
if google_api_key:
    print(f"google api key starts with {google_api_key[:8]}")
else:
    print(f"google api key is not found")

openai api key starts with sk-proj-
anthropic api key starts with sk-a
google api key starts with AIzaSyDC


In [40]:
openai = OpenAI()
claude = anthropic.Anthropic()

AttributeError: 'Anthropic' object has no attribute 'Anthropic'

In [35]:
class ConversationManager:
    def __init__(self):
        self.conversation_history=[]
        self.participants={}
    
    def add_participant(self,name,chatbot):
        """Adding Model to the Participant"""
        self.participants[name]=chatbot
    
    def add_message(self,speaker,message):
        """Add message to conversation history"""
        self.conversation_history.append({
            "speaker": speaker,
            "role": "assistant" if speaker in self.participants else "user",
            "content": message
        })
    def get_context_for_model(self,model_name):
        """Convert the shared history to model specific format"""
        messages =[]
        for msg in self.conversation_history:
            if msg["speaker"] == model_name:
                messages.append({"role":"assistant","content":msg["content"]})
            else:
                messages.append({"role":"user","content":msg["content"]})
        return messages
    def run_conversation(self,starting_message,turns=3,round_robin=True):
        """run a multi-model conversation for specific number of turns"""
        current_message = starting_message
        models = list(self.participants.keys())

        # add initial user message so models have context
        self.add_message("user", starting_message)

        for _ in range(turns):
            for model_name in models:
                model_context = self.get_context_for_model(model_name)
                chatbot=self.participants[model_name]
                response=chatbot.generate_response(model_context)
                self.add_message(model_name,response)
                print(f"{model_name}:\n{response}\n")
                
                if not round_robin:
                    # If not round-robin, use this response as input to next model
                    current_message = response

             

In [ ]:
class ChatBot:
    def __init__(self,model_name,system_prompt,**kwargs):
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.api_key = kwargs.get('api_key', None)
        self.base_url = kwargs.get('base_url', None)

    def generate_response(self,messages):
        """generate a response based on the messages and don't provide history"""
        full_messages = [{"role":"system","content":self.system_prompt}] + messages
        try:
            if 'claude' in self.model_name.lower():
                # use the shared claude_client created earlier
                claude_messages = [m for m in messages if m['role']!='system']
                response = claude.messages.create(
                    model=self.model_name,
                    system=self.system_prompt,
                    messages=claude_messages,
                    max_tokens=200
                )
                return response.content[0].text
            else:
                # use OpenAI-style chat completions
                client = OpenAI(api_key=self.api_key, base_url=self.base_url) if (self.api_key or self.base_url) else openai
                response = client.chat.completions.create(
                    model=self.model_name,
                    messages=full_messages
                )
                return response.choices[0].message.content
    
        except Exception as e:
            return f"Error Str{e}"






In [37]:
gpt_bot = ChatBot("gpt-4.1-mini", "You are witty and sarcastic.")
claude_bot = ChatBot("claude-3-5-haiku-20241022", "You are thoughtful and philosophical.")

# Optional: local Ollama model (only if installed). Comment out if not available.
# model_name = "llama3.2:latest"
# system_prompt = "You are a helpful assistant that is very argumentative in a snarky way."
# kwargs = {
#     "api_key": "ollama",
#     "base_url": 'http://localhost:11434/v1'
# }
# qwen = ChatBot(model_name, system_prompt, **kwargs)

# Set up conversation manager
conversation = ConversationManager()
conversation.add_participant("GPT", gpt_bot)
conversation.add_participant("Claude", claude_bot)
# conversation.add_participant("Qwen", qwen)  # enable only if the model exists

conversation.run_conversation("What's the most interesting technology trend right now?", turns=2)

GPT:
Oh, you mean aside from cat videos taking over the internet again? Alright, if I have to pick a “most interesting” tech trend right now, I’d say it’s probably generative AI. It’s like the tech version of your friend who never stops talking—but instead of annoying you at parties, it can write poetry, code, and even create art. It’s both amazing and mildly terrifying, kind of like a digital Frankenstein that’s learned to do your job... and maybe your therapist’s, too.

Claude:
Error Strname 'claude_client' is not defined

GPT:
Ah, the classic "Strname 'claude_client' is not defined" error—sounds like your code just ghosted you. This usually means you tried to use `claude_client` without actually introducing it first, kind of like inviting someone to a party and then forgetting to tell them where it is.

Here’s the usual recipe to fix that:

1. Make sure you’ve imported or defined `claude_client` before using it.
2. If `claude_client` is part of a library, double-check that you insta